<a href="https://colab.research.google.com/github/hsallrounder/Data_Mining_And_Predictive_Analysis-Workshop-2022/blob/main/tregression_(AirPassengers).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#to install in anaconda environment
#conda install -c conda-forge pycaret

In [2]:
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# read csv file
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/Data Mining And Predictive Analysis Workshop/AirPassengers.csv")
data['Date'] = pd.to_datetime(data['Month'])
data.drop(['Month'], axis=1, inplace=True)
data.head()

,#Passengers,Date
0,112,1949-01-01
1,118,1949-02-01
2,132,1949-03-01
3,129,1949-04-01
4,121,1949-05-01


In [5]:
# create 12 month moving average
data['MA12'] = data['#Passengers'].rolling(12).mean()
# plot the data and MA
import plotly.express as px
fig = px.line(data, x="Date", y=["#Passengers", "MA12"], template = 'plotly_dark')
fig.show()

In [6]:
# extract month and year from dates
data['Month'] = [i.month for i in data['Date']]
data['Year'] = [i.year for i in data['Date']]
# create a sequence of numbers
data['Series'] = np.arange(1,len(data)+1)
# drop unnecessary columns and re-arrange
data.drop(['Date', 'MA12'], axis=1, inplace=True)
data = data[['Series', 'Year', 'Month', '#Passengers']] 
# check the head of the dataset
data.head()

,Series,Year,Month,#Passengers
0,1,1949,1,112
1,2,1949,2,118
2,3,1949,3,132
3,4,1949,4,129
4,5,1949,5,121


In [7]:
# split data into train-test set
train = data[data['Year'] < 1960]
test = data[data['Year'] >= 1960]

In [8]:
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
# import the regression module
from pycaret.regression import *

# initialize setup
s = setup(data = train, test_data = test, target = '#Passengers', fold_strategy = 'timeseries', numeric_features = ['Year', 'Series'], fold = 3, transform_target = True, session_id = 123)

,Description,Value
0,session_id,123
1,Target,#Passengers
2,Original Data,"(132, 4)"
3,Missing Values,False
4,Numeric Features,2
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(132, 13)"


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=['Year', 'Series'],
                                      target='#Passengers', time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=No...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
                ('dummy', Dummify(target='#Pas

In [10]:
best = compare_models(sort = 'MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lar,Least Angle Regression,22.3980,923.8651,28.2855,0.5621,0.0878,0.0746,0.0200
lr,Linear Regression,22.3981,923.8747,28.2856,0.5621,0.0878,0.0746,0.8867
huber,Huber Regressor,22.4244,892.2573,27.9475,0.5981,0.0880,0.0749,0.0333
br,Bayesian Ridge,22.4783,932.2165,28.5483,0.5611,0.0884,0.0746,0.0167
ridge,Ridge Regression,23.1976,1003.9423,30.0409,0.5258,0.0933,0.0764,0.0133
lasso,Lasso Regression,38.4188,2413.5096,46.8468,0.0882,0.1473,0.1241,0.0167
en,Elastic Net,40.6486,2618.8753,49.4048,-0.0824,0.1563,0.1349,0.0167
omp,Orthogonal Matching Pursuit,44.3054,3048.2658,53.8613,-0.4499,0.1713,0.1520,0.0167
gbr,Gradient Boosting Regressor,50.1217,4032.0567,61.2306,-0.6189,0.2034,0.1538,0.0333
rf,Random Forest Regressor,52.7754,4705.6863,65.6728,-0.7962,0.2148,0.1592,0.2500


INFO:logs:create_model_container: 18
INFO:logs:master_model_container: 18
INFO:logs:display_container: 2
INFO:logs:PowerTransformedTargetRegressor(copy_X=True, eps=2.220446049250313e-16,
                                fit_intercept=True, fit_path=True, jitter=None,
                                n_nonzero_coefs=500, normalize=True,
                                power_transformer_method='box-cox',
                                power_transformer_standardize=True,
                                precompute='auto', random_state=123,
                                regressor=Lars(copy_X=True,
                                               eps=2.220446049250313e-16,
                                               fit_intercept=True,
                                               fit_path=True, jitter=None,
                                               n_nonzero_coefs=500,
                                               normalize=True,
                                               preco

In [11]:
prediction_holdout = predict_model(best)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=PowerTransformedTargetRegressor(copy_X=True, eps=2.220446049250313e-16,
                                fit_intercept=True, fit_path=True, jitter=None,
                                n_nonzero_coefs=500, normalize=True,
                                power_transformer_method='box-cox',
                                power_transformer_standardize=True,
                                precompute='auto', random_state=123,
                                regressor=Lars(copy_X=True,
                                               eps=2.220446049250313e-16,
                                               fit_intercept=True,
                                               fit_path=True, jitter=None,
                                               n_nonzero_coefs=500,
                                               normalize=True,
                                               precompute='auto',
                            

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Least Angle Regression,25.0714,972.2733,31.1813,0.8245,0.0692,0.0571


In [12]:
# generate predictions on the original dataset
predictions = predict_model(best, data=data)
# add a date column in the dataset
predictions['Date'] = pd.date_range(start='1949-01-01', end = '1960-12-01', freq = 'MS')
# line plot
fig = px.line(predictions, x='Date', y=["#Passengers", "Label"], template = 'plotly_dark')
# add a vertical rectange for test-set separation
fig.add_vrect(x0="1960-01-01", x1="1960-12-01", fillcolor="grey", opacity=0.25, line_width=0)
fig.show()

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=PowerTransformedTargetRegressor(copy_X=True, eps=2.220446049250313e-16,
                                fit_intercept=True, fit_path=True, jitter=None,
                                n_nonzero_coefs=500, normalize=True,
                                power_transformer_method='box-cox',
                                power_transformer_standardize=True,
                                precompute='auto', random_state=123,
                                regressor=Lars(copy_X=True,
                                               eps=2.220446049250313e-16,
                                               fit_intercept=True,
                                               fit_path=True, jitter=None,
                                               n_nonzero_coefs=500,
                                               normalize=True,
                                               precompute='auto',
                            

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Least Angle Regression,12.5381,278.7548,16.696,0.9805,0.0538,0.0447


In [13]:
final_best = finalize_model(best)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=PowerTransformedTargetRegressor(copy_X=True, eps=2.220446049250313e-16,
                                fit_intercept=True, fit_path=True, jitter=None,
                                n_nonzero_coefs=500, normalize=True,
                                power_transformer_method='box-cox',
                                power_transformer_standardize=True,
                                precompute='auto', random_state=123,
                                regressor=Lars(copy_X=True,
                                               eps=2.220446049250313e-16,
                                               fit_intercept=True,
                                               fit_path=True, jitter=None,
                                               n_nonzero_coefs=500,
                                               normalize=True,
                                               precompute='auto',
                          

In [14]:
future_dates = pd.date_range(start = '1961-01-01', end = '1965-01-01', freq = 'MS')
future_df = pd.DataFrame()
future_df['Month'] = [i.month for i in future_dates]
future_df['Year'] = [i.year for i in future_dates]    
future_df['Series'] = np.arange(145,(145+len(future_dates)))
future_df.head()

,Month,Year,Series
0,1,1961,145
1,2,1961,146
2,3,1961,147
3,4,1961,148
4,5,1961,149


In [15]:
predictions_future = predict_model(final_best, data=future_df)
predictions_future.head()

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=PowerTransformedTargetRegressor(copy_X=True, eps=2.220446049250313e-16,
                                fit_intercept=True, fit_path=True, jitter=None,
                                n_nonzero_coefs=500, normalize=True,
                                power_transformer_method='box-cox',
                                power_transformer_standardize=True,
                                precompute='auto', random_state=123,
                                regressor=Lars(copy_X=True,
                                               eps=2.220446049250313e-16,
                                               fit_intercept=True,
                                               fit_path=True, jitter=None,
                                               n_nonzero_coefs=500,
                                               normalize=True,
                                               precompute='auto',
                            

,Month,Year,Series,Label
0,1,1961,145,486.278268
1,2,1961,146,482.208187
2,3,1961,147,550.485967
3,4,1961,148,535.187177
4,5,1961,149,538.923789


In [16]:
concat_df = pd.concat([data,predictions_future], axis=0)
concat_df_i = pd.date_range(start='1949-01-01', end = '1965-01-01', freq = 'MS')
concat_df.set_index(concat_df_i, inplace=True)
fig = px.line(concat_df, x=concat_df.index, y=["#Passengers", "Label"], template = 'plotly_dark')
fig.show()